# High Accuracy CNN model

## Import libraries

In [0]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


## Build CNN model

In [0]:
classifier = Sequential()
classifier.add(Convolution2D(filters = 32, 
                             kernel_size = (3, 3), 
                             activation = 'relu', 
                             input_shape = (128, 128, 3),
                             data_format='channels_last'))
classifier.add(Convolution2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Convolution2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
classifier.add(Convolution2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Convolution2D(filters = 128, kernel_size = (3, 3), activation = 'relu'))
classifier.add(Convolution2D(filters = 128, kernel_size = (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate = 0.2))
classifier.add(Dense(units = 64, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 58, 58, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxP

## Training the model

In [0]:
train_datagen = ImageDataGenerator(rescale = 1.0/255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True, 
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale = 1.0/255)

training_set = train_datagen.flow_from_directory('data/training_set', 
                                                 target_size = (128, 128), 
                                                 batch_size = 32, 
                                                 class_mode = 'binary', 
                                                 subset = 'training')

validation_set = train_datagen.flow_from_directory('data/training_set', 
                                                 target_size = (128, 128), 
                                                 batch_size = 32, 
                                                 class_mode = 'binary',
                                                   subset = 'validation')

test_set = test_datagen.flow_from_directory('data/test_set', 
                                            target_size = (128, 128), 
                                            batch_size = 32, 
                                            class_mode = 'binary')

Found 6400 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 6400/32,
                         epochs = 100,
                         validation_data = validation_set,
                         validation_steps = 1600/32,
                         verbose = 1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
200/200 [==============================] - 50s 249ms/step - loss: 0.7054 - acc: 0.5100 - val_loss: 0.6894 - val_acc: 0.5100
Epoch 2/100
200/200 [==============================] - 46s 228ms/step - loss: 0.7000 - acc: 0.5508 - val_loss: 0.6822 - val_acc: 0.5594
Epoch 3/100
200/200 [==============================] - 44s 221ms/step - loss: 0.6770 - acc: 0.5875 - val_loss: 0.6456 - val_acc: 0.6106
Epoch 4/100
200/200 [==============================] - 47s 233ms/step - loss: 0.6682 - acc: 0.6289 - val_loss: 0.6195 - val_acc: 0.6600
Epoch 5/100
200/200 [==============================] - 44s 221ms/step - loss: 0.6198 - acc: 0.6650 - val_loss: 0.5550 - val_acc: 0.7131
Epoch 6/100
200/200 [==============================] - 46s 228ms/step - loss: 0.5724 - acc: 0.7144 - val_loss: 0.5102 - val_acc: 0.7681
Epoch 7/100
200/200 [==============================] - 44s 221ms/step - loss: 0.5406 - acc: 0.7314 - val_loss: 0.4596 - val_acc: 0.7981


## Evaluating the model

In [5]:
print("Metrics: {}".format(classifier.metrics_names))
metrics = classifier.evaluate_generator(test_set, steps=2000/32, verbose=1)
print("Test Loss: {} and Accuracy: {}%".format(metrics[0], metrics[1]*100))

Metrics: ['loss', 'acc']
63/62 [==============================] - 5s 85ms/step
Test Loss: 0.5233282656669617 and Accuracy: 85.39999999999999%
